In [2]:
import pymongo
from collections import defaultdict

In [3]:
# print all available databases
client = pymongo.MongoClient('localhost', 27017)
cursor = client.list_databases()
for db in cursor:
    print(db)

{'name': '10000', 'sizeOnDisk': 270336.0, 'empty': False}
{'name': '10000_cl', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_clagreement', 'sizeOnDisk': 253952.0, 'empty': False}
{'name': '10000_cl_claudience', 'sizeOnDisk': 368640.0, 'empty': False}
{'name': '10000_cl_clcontroversial', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_cldisagreement', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_clinformative', 'sizeOnDisk': 258048.0, 'empty': False}
{'name': '10000_cl_clmean', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_clpersuasive', 'sizeOnDisk': 360448.0, 'empty': False}
{'name': '10000_cl_clsentiment', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_cl_cltopic', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_ner', 'sizeOnDisk': 274432.0, 'empty': False}
{'name': '10000_ner_cl', 'sizeOnDisk': 262144.0, 'empty': False}
{'name': '10000_ner_cl_clagreement', 'sizeOnDisk': 253952.0, 'empty': False}
{'name': '10000_ner_

In [4]:
def get_best(db, metric, config_param=None, config_val=None, config_param2=None, config_val2=None, one=True):
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient[db]
    res = mydb["metrics"].aggregate([{
        "$match": {"name": metric}  # only consider metric
    },
        {"$unwind": "$values"},
        {"$group":
         {'_id': '$_id',
          'val': {'$max': "$values"}, 'run_id' : { '$first': '$run_id' }}
         },  # find min values
        {"$sort": {"val": -1}}  # sort
    ])
    
    if config_param is None:
        if one:
            return list(res)[0]['val']
        return [x['val'] for x in list(res)]
    
    filtered_res = []
#   filter only results for the config
    for res_obj in res:
        run = list(mydb['runs'].find({'_id': res_obj['run_id']}))[0]
        if config_param in run['config']:
            if run['config'][config_param] == config_val:
                if config_param2 is None:
                    filtered_res.append(res_obj)
                else:
                    if not config_param2 in run['config']:
                        continue
#                     print(run['config'][config_param2])
                    if run['config'][config_param2] == config_val2:
#                         print('yes')
                        filtered_res.append(res_obj)
        else:
            pass
#             print('NB: just used run without considering param')
#             filtered_res.append(res_obj)
    if one:
        return filtered_res[0]['val']
    return [x['val'] for x in filtered_res]

In [3]:
all_cols =  ['claudience', 'clpersuasive', 'clsentiment', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic']

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['30000_ner_cl_cltopic']
for x in mydb['runs'].find({}):
    print(x['_id'])

def get_best(db, metric, config_param=None, config_val=None, config_param2=None, config_val2=None, one=True):
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient[db]
    res = mydb["metrics"].aggregate([{
        "$match": {"name": metric}  # only consider metric
    },
        {"$unwind": "$values"},
        {"$group":
         {'_id': '$_id',
          'val': {'$max': "$values"}, 'run_id' : { '$first': '$run_id' }}
         },  # find min values
        {"$sort": {"val": -1}}  # sort
    ])
    
    if config_param is None:
        if one:
            return list(res)[0]['val']
        return [x['val'] for x in list(res)]
    
    filtered_res = []
#   filter only results for the config
    for res_obj in res:
        run = list(mydb['runs'].find({'_id': res_obj['run_id']}))[0]
        if config_param in run['config']:
            if run['config'][config_param] == config_val:
                if config_param2 is None:
                    filtered_res.append(res_obj)
                else:
                    if not config_param2 in run['config']:
                        continue
#                     print(run['config'][config_param2])
                    if run['config'][config_param2] == config_val2:
#                         print('yes')
                        filtered_res.append(res_obj)
        else:
            pass
#             print('NB: just used run without considering param')
#             filtered_res.append(res_obj)
    if one:
        return filtered_res[0]['val']
    return [x['val'] for x in filtered_res]

1
2
3
4
5
6
7
8
9
10


In [4]:
def compare(exps, best, **kwargs):
    best_exps = defaultdict(lambda: 0)
    if not type(exps) is list:
        exps = [exps]
        
    for column in ['claudience', 'clpersuasive', 'clsentiment', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic']:
        print('#', column)
        print()
        for met in ['accuracy', 'F1_macro']:
#         for met in ['accuracy']:
            print(met)
            for x in [str(e) + '_cl_' + column for e in exps]:
#                 try:
                best_bla = get_best(x, met, **kwargs)
#                 except:
#                     print('skip :', x)
#                     continue
                best_exps[column+met] = best_bla
                
                sin_best = best[column+met]
                
                if best_bla > sin_best:
                    best_bla = '* ' + str(best_bla)
                else:
                    if best_bla < sin_best:
                        sin_best = '* ' + str(sin_best)
                print(x[:20],'\t',best_bla , '\t', sin_best)
            print()
    return best_exps

In [5]:
# compare_ner(30000)

# compare_ner(20000)

# compare_ner(10000)

In [6]:
best = defaultdict(lambda: 0)

In [7]:
sizes=[10000, 20000, 30000, 40000, 50000, 60000]
for column in ['claudience', 'clpersuasive', 'clsentiment', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic']:
    print(column)
    for met in ['accuracy', 'F1_macro', 'F1_weighted']:
#         for met in ['accuracy']:
        print(met)
        for x in [str(size) + '_cl_' + column for size in sizes]:
            the_best = get_best(x, met)
            key = column + met
            best[key] = max(best[key], the_best)
            print(x[:20],'\t', the_best)
        print()

claudience
accuracy
10000_cl_claudience 	 0.7745266556739807
20000_cl_claudience 	 0.7246127128601074
30000_cl_claudience 	 0.7710843086242676
40000_cl_claudience 	 0.7624784708023071
50000_cl_claudience 	 0.7469879388809204
60000_cl_claudience 	 0.7624784708023071

F1_macro
10000_cl_claudience 	 0.7290623208990556
20000_cl_claudience 	 0.6436392548841435
30000_cl_claudience 	 0.7249258677830106
40000_cl_claudience 	 0.7090361445783133
50000_cl_claudience 	 0.6883604505632039
60000_cl_claudience 	 0.7071830603692856

F1_weighted
10000_cl_claudience 	 0.7933541017653167
20000_cl_claudience 	 0.7933541017653167
30000_cl_claudience 	 0.7933541017653167
40000_cl_claudience 	 0.7933541017653167
50000_cl_claudience 	 0.7933541017653167
60000_cl_claudience 	 0.7933541017653167

clpersuasive
accuracy
10000_cl_clpersuasiv 	 0.8559176921844482
20000_cl_clpersuasiv 	 0.8576329350471497
30000_cl_clpersuasiv 	 0.8593481779098511
40000_cl_clpersuasiv 	 0.854202389717102
50000_cl_clpersuasiv 	 0.8559

In [8]:
# for b in best.items():
#     print(b)

In [9]:
sizes=['30000_threads']
for column in ['claudience', 'clpersuasive', 'clsentiment', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic']:
    print(column)
    for met in ['accuracy', 'F1_macro', 'F1_weighted']:
#         for met in ['accuracy']:
        print(met)
        for x in [str(size) + '_cl_' + column for size in sizes]:
            print(x[:20],'\t', get_best(x, met), '\t', best[column+met])
        print()

claudience
accuracy
30000_threads_cl_cla 	 0.8278829455375671 	 0.7745266556739807

F1_macro
30000_threads_cl_cla 	 0.7743443947303008 	 0.7290623208990556

F1_weighted
30000_threads_cl_cla 	 0.8468011815024025 	 0.7933541017653167

clpersuasive
accuracy
30000_threads_cl_clp 	 0.8679245114326477 	 0.8593481779098511

F1_macro
30000_threads_cl_clp 	 0.6509468114731273 	 0.5925756789482789

F1_weighted
30000_threads_cl_clp 	 0.9213691026827013 	 0.9213691026827013

clsentiment
accuracy
30000_threads_cl_cls 	 0.6297577619552612 	 0.6366782188415527

F1_macro
30000_threads_cl_cls 	 0.4144837856651358 	 0.42310453521081576

F1_weighted
30000_threads_cl_cls 	 0.6937853107344633 	 0.6989928867872834

clagreement
accuracy
30000_threads_cl_cla 	 0.8919382691383362 	 0.9108061790466309

F1_macro
30000_threads_cl_cla 	 0.6499812382739212 	 0.7289772727272726

F1_weighted
30000_threads_cl_cla 	 0.9380692167577412 	 0.9380692167577412

cldisagreement
accuracy
30000_threads_cl_cld 	 0.75300168991088

In [10]:
compare('30000_threads', best=best)

# claudience

accuracy
30000_threads_cl_cla 	 * 0.8278829455375671 	 0.7745266556739807

F1_macro
30000_threads_cl_cla 	 * 0.7743443947303008 	 0.7290623208990556

# clpersuasive

accuracy
30000_threads_cl_clp 	 * 0.8679245114326477 	 0.8593481779098511

F1_macro
30000_threads_cl_clp 	 * 0.6509468114731273 	 0.5925756789482789

# clsentiment

accuracy
30000_threads_cl_cls 	 0.6297577619552612 	 * 0.6366782188415527

F1_macro
30000_threads_cl_cls 	 0.4144837856651358 	 * 0.42310453521081576

# clagreement

accuracy
30000_threads_cl_cla 	 0.8919382691383362 	 * 0.9108061790466309

F1_macro
30000_threads_cl_cla 	 0.6499812382739212 	 * 0.7289772727272726

# cldisagreement

accuracy
30000_threads_cl_cld 	 0.7530016899108887 	 * 0.7598627805709839

F1_macro
30000_threads_cl_cld 	 * 0.7451566955802895 	 0.7430602110490377

# clinformative

accuracy
30000_threads_cl_cli 	 0.8610634803771973 	 * 0.8627787232398987

F1_macro
30000_threads_cl_cli 	 0.6575861363531376 	 * 0.671234421699656

# clm

defaultdict(<function __main__.compare.<locals>.<lambda>()>,
            {'claudienceaccuracy': 0.8278829455375671,
             'claudienceF1_macro': 0.7743443947303008,
             'clpersuasiveaccuracy': 0.8679245114326477,
             'clpersuasiveF1_macro': 0.6509468114731273,
             'clsentimentaccuracy': 0.6297577619552612,
             'clsentimentF1_macro': 0.4144837856651358,
             'clagreementaccuracy': 0.8919382691383362,
             'clagreementF1_macro': 0.6499812382739212,
             'cldisagreementaccuracy': 0.7530016899108887,
             'cldisagreementF1_macro': 0.7451566955802895,
             'clinformativeaccuracy': 0.8610634803771973,
             'clinformativeF1_macro': 0.6575861363531376,
             'clmeanaccuracy': 0.838765025138855,
             'clmeanF1_macro': 0.7292330148619958,
             'clcontroversialaccuracy': 0.7358490824699402,
             'clcontroversialF1_macro': 0.67836902552786,
             'cltopicaccuracy': 0.7735

In [11]:
compare('30000_threads_last', best=best)

# claudience

accuracy
30000_threads_last_c 	 * 0.824440598487854 	 0.7745266556739807

F1_macro
30000_threads_last_c 	 * 0.7688112477373448 	 0.7290623208990556

# clpersuasive

accuracy
30000_threads_last_c 	 0.8576329350471497 	 * 0.8593481779098511

F1_macro
30000_threads_last_c 	 * 0.6074769666902906 	 0.5925756789482789

# clsentiment

accuracy
30000_threads_last_c 	 * 0.6384083032608032 	 0.6366782188415527

F1_macro
30000_threads_last_c 	 0.42145118535480797 	 * 0.42310453521081576

# clagreement

accuracy
30000_threads_last_c 	 0.9108061790466309 	 0.9108061790466309

F1_macro
30000_threads_last_c 	 0.7142757123853531 	 * 0.7289772727272726

# cldisagreement

accuracy
30000_threads_last_c 	 0.7272727489471436 	 * 0.7598627805709839

F1_macro
30000_threads_last_c 	 0.7245464675729887 	 * 0.7430602110490377

# clinformative

accuracy
30000_threads_last_c 	 0.8593481779098511 	 * 0.8627787232398987

F1_macro
30000_threads_last_c 	 0.6319366119753429 	 * 0.671234421699656

# clmea

defaultdict(<function __main__.compare.<locals>.<lambda>()>,
            {'claudienceaccuracy': 0.824440598487854,
             'claudienceF1_macro': 0.7688112477373448,
             'clpersuasiveaccuracy': 0.8576329350471497,
             'clpersuasiveF1_macro': 0.6074769666902906,
             'clsentimentaccuracy': 0.6384083032608032,
             'clsentimentF1_macro': 0.42145118535480797,
             'clagreementaccuracy': 0.9108061790466309,
             'clagreementF1_macro': 0.7142757123853531,
             'cldisagreementaccuracy': 0.7272727489471436,
             'cldisagreementF1_macro': 0.7245464675729887,
             'clinformativeaccuracy': 0.8593481779098511,
             'clinformativeF1_macro': 0.6319366119753429,
             'clmeanaccuracy': 0.8421955108642578,
             'clmeanF1_macro': 0.7178834420366085,
             'clcontroversialaccuracy': 0.7204116582870483,
             'clcontroversialF1_macro': 0.6562657769775282,
             'cltopicaccuracy': 0.7

In [12]:
 def prev3(sizes=['30000_threads_prev3']):
    for column in ['claudience', 'clpersuasive', 'clsentiment', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic']:
        print(column)
        for met in ['accuracy', 'F1_macro', 'F1_weighted']:
#         for met in ['accuracy']:
            print(met)
            for x in [str(size) + '_cl_' + column for size in sizes]:
                print(x[:20],'\t', get_best(x, met))
            print()

In [13]:
prev3()

claudience
accuracy
30000_threads_prev3_ 	 0.826161801815033

F1_macro
30000_threads_prev3_ 	 0.7745440434613892

F1_weighted
30000_threads_prev3_ 	 0.8443500790324575

clpersuasive
accuracy
30000_threads_prev3_ 	 0.8713550567626953

F1_macro
30000_threads_prev3_ 	 0.6216742375081116

F1_weighted
30000_threads_prev3_ 	 0.9213691026827013

clsentiment
accuracy
30000_threads_prev3_ 	 0.6176470518112183

F1_macro
30000_threads_prev3_ 	 0.3828716776319055

F1_weighted
30000_threads_prev3_ 	 0.6900196629901889

clagreement
accuracy
30000_threads_prev3_ 	 0.9022298455238342

F1_macro
30000_threads_prev3_ 	 0.6961155007653166

F1_weighted
30000_threads_prev3_ 	 0.9380692167577412

cldisagreement
accuracy
30000_threads_prev3_ 	 0.6672384142875671

F1_macro
30000_threads_prev3_ 	 0.6596593644679827

F1_weighted
30000_threads_prev3_ 	 0.6649734162695073

clinformative
accuracy
30000_threads_prev3_ 	 0.8473413586616516

F1_macro
30000_threads_prev3_ 	 0.6258907764531054

F1_weighted
30000_threads

In [14]:
def prev6(sizes=['30000_threads_prev6']):
    for column in ['claudience', 'clpersuasive', 'clsentiment', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic']:
        print(column)
        for met in ['accuracy', 'F1_macro', 'F1_weighted']:
#         for met in ['accuracy']:
            print(met)
            for x in [str(size) + '_cl_' + column for size in sizes]:
                print(x[:20],'\t', get_best(x, met))
            print()

In [15]:
prev6()

claudience
accuracy
30000_threads_prev6_ 	 0.8296041488647461

F1_macro
30000_threads_prev6_ 	 0.7790085178482924

F1_weighted
30000_threads_prev6_ 	 0.8474615700348712

clpersuasive
accuracy
30000_threads_prev6_ 	 0.8593481779098511

F1_macro
30000_threads_prev6_ 	 0.5357651124966115

F1_weighted
30000_threads_prev6_ 	 0.9213691026827013

clsentiment
accuracy
30000_threads_prev6_ 	 0.621107280254364

F1_macro
30000_threads_prev6_ 	 0.39410931755300505

F1_weighted
30000_threads_prev6_ 	 0.690398235418735

clagreement
accuracy
30000_threads_prev6_ 	 0.8970840573310852

F1_macro
30000_threads_prev6_ 	 0.672239739923248

F1_weighted
30000_threads_prev6_ 	 0.9380692167577412

cldisagreement
accuracy
30000_threads_prev6_ 	 0.7084047794342041

F1_macro
30000_threads_prev6_ 	 0.7069554830163686

F1_weighted
30000_threads_prev6_ 	 0.705859655904911

clinformative
accuracy
30000_threads_prev6_ 	 0.8593481779098511

F1_macro
30000_threads_prev6_ 	 0.654301168575726

F1_weighted
30000_threads_pr

In [16]:
sizes=['30000_ctx_ner_fixed']
for column in ['claudience', 'clpersuasive', 'clsentiment', 'clagreement', 'cldisagreement', 'clinformative', 'clmean', 'clcontroversial', 'cltopic']:
    print(column)
    for met in ['accuracy', 'F1_macro', 'F1_weighted']:
#         for met in ['accuracy']:
        print(met)
        for x in [str(size) + '_cl_' + column for size in sizes]:
            print(x[:20],'\t', get_best(x, met), '\t', best[column+met])
        print()

claudience
accuracy
30000_ctx_ner_fixed_ 	 0.7565217614173889 	 0.7745266556739807

F1_macro
30000_ctx_ner_fixed_ 	 0.7009284773635227 	 0.7290623208990556

F1_weighted
30000_ctx_ner_fixed_ 	 0.7890016215276024 	 0.7933541017653167

clpersuasive
accuracy
30000_ctx_ner_fixed_ 	 0.8578855991363525 	 0.8593481779098511

F1_macro
30000_ctx_ner_fixed_ 	 0.5974686067370938 	 0.5925756789482789

F1_weighted
30000_ctx_ner_fixed_ 	 0.9204864359214221 	 0.9213691026827013

clsentiment
accuracy
30000_ctx_ner_fixed_ 	 0.6293706297874451 	 0.6366782188415527

F1_macro
30000_ctx_ner_fixed_ 	 0.41121576108803465 	 0.42310453521081576

F1_weighted
30000_ctx_ner_fixed_ 	 0.692711534090686 	 0.6989928867872834

clagreement
accuracy
30000_ctx_ner_fixed_ 	 0.9081456065177917 	 0.9108061790466309

F1_macro
30000_ctx_ner_fixed_ 	 0.7016069128280349 	 0.7289772727272726

F1_weighted
30000_ctx_ner_fixed_ 	 0.9373848987108655 	 0.9380692167577412

cldisagreement
accuracy
30000_ctx_ner_fixed_ 	 0.72963607311248

In [17]:
# no finetuning
compare('dat_false_par_false_hea_false30000', best=best)

# claudience

accuracy
dat_false_par_false_ 	 0.6609294414520264 	 * 0.7745266556739807

F1_macro
dat_false_par_false_ 	 0.4440351796731118 	 * 0.7290623208990556

# clpersuasive

accuracy
dat_false_par_false_ 	 0.854202389717102 	 * 0.8593481779098511

F1_macro
dat_false_par_false_ 	 0.5057865251363703 	 * 0.5925756789482789

# clsentiment

accuracy
dat_false_par_false_ 	 0.5415225028991699 	 * 0.6366782188415527

F1_macro
dat_false_par_false_ 	 0.20176751145609276 	 * 0.42310453521081576

# clagreement

accuracy
dat_false_par_false_ 	 0.8833619356155396 	 * 0.9108061790466309

F1_macro
dat_false_par_false_ 	 0.48654550218722026 	 * 0.7289772727272726

# cldisagreement

accuracy
dat_false_par_false_ 	 0.6260720491409302 	 * 0.7598627805709839

F1_macro
dat_false_par_false_ 	 0.5921858292271804 	 * 0.7430602110490377

# clinformative

accuracy
dat_false_par_false_ 	 0.8353344798088074 	 * 0.8627787232398987

F1_macro
dat_false_par_false_ 	 0.4955685990225195 	 * 0.671234421699656

# cl

defaultdict(<function __main__.compare.<locals>.<lambda>()>,
            {'claudienceaccuracy': 0.6609294414520264,
             'claudienceF1_macro': 0.4440351796731118,
             'clpersuasiveaccuracy': 0.854202389717102,
             'clpersuasiveF1_macro': 0.5057865251363703,
             'clsentimentaccuracy': 0.5415225028991699,
             'clsentimentF1_macro': 0.20176751145609276,
             'clagreementaccuracy': 0.8833619356155396,
             'clagreementF1_macro': 0.48654550218722026,
             'cldisagreementaccuracy': 0.6260720491409302,
             'cldisagreementF1_macro': 0.5921858292271804,
             'clinformativeaccuracy': 0.8353344798088074,
             'clinformativeF1_macro': 0.4955685990225195,
             'clmeanaccuracy': 0.7941681146621704,
             'clmeanF1_macro': 0.4948387582881254,
             'clcontroversialaccuracy': 0.7015437483787537,
             'clcontroversialF1_macro': 0.5972445440325299,
             'cltopicaccuracy': 0.

In [18]:
# lm_model_type='trained_0_embed_prevent'

In [19]:
dontembed6best = compare('only_threads_unlimited_30000', config_param='embed_prevent', config_val=6, config_param2='lm_model_type', config_val2='trained_6_embed_prevent', best=best)

# claudience

accuracy
only_threads_unlimit 	 * 0.8313252925872803 	 0.7745266556739807

F1_macro
only_threads_unlimit 	 * 0.7807716001848144 	 0.7290623208990556

# clpersuasive

accuracy
only_threads_unlimit 	 * 0.8644939661026001 	 0.8593481779098511

F1_macro
only_threads_unlimit 	 * 0.6782121512466394 	 0.5925756789482789

# clsentiment

accuracy
only_threads_unlimit 	 0.6159169673919678 	 * 0.6366782188415527

F1_macro
only_threads_unlimit 	 0.40400866190474294 	 * 0.42310453521081576

# clagreement

accuracy
only_threads_unlimit 	 0.9108061790466309 	 0.9108061790466309

F1_macro
only_threads_unlimit 	 0.7177890016348674 	 * 0.7289772727272726

# cldisagreement

accuracy
only_threads_unlimit 	 0.728987991809845 	 * 0.7598627805709839

F1_macro
only_threads_unlimit 	 0.7187040119659356 	 * 0.7430602110490377

# clinformative

accuracy
only_threads_unlimit 	 0.8576329350471497 	 * 0.8627787232398987

F1_macro
only_threads_unlimit 	 0.671010498687664 	 * 0.671234421699656

# clmean

In [ ]:
# dontembed0best = compare('only_threads_unlimited_30000', config_param='embed_prevent', config_param2='lm_model_type', config_val2='trained_0_embed_prevent', config_val=0, best=dontembed6best)

In [22]:
dontembed0best = compare('only_threads_unlimited_30000', config_param='embed_prevent', config_param2='lm_model_type', config_val2='trained_0_embed_prevent', config_val=0, best=best)

# claudience

accuracy
only_threads_unlimit 	 * 0.8278829455375671 	 0.7745266556739807

F1_macro
only_threads_unlimit 	 * 0.7743443947303008 	 0.7290623208990556

# clpersuasive

accuracy
only_threads_unlimit 	 0.8576329350471497 	 * 0.8593481779098511

F1_macro
only_threads_unlimit 	 * 0.6851323200370342 	 0.5925756789482789

# clsentiment

accuracy
only_threads_unlimit 	 0.6228373646736145 	 * 0.6366782188415527

F1_macro
only_threads_unlimit 	 0.4224306339335213 	 * 0.42310453521081576

# clagreement

accuracy
only_threads_unlimit 	 * 0.9193825125694275 	 0.9108061790466309

F1_macro
only_threads_unlimit 	 * 0.7452610049737369 	 0.7289772727272726

# cldisagreement

accuracy
only_threads_unlimit 	 0.7204116582870483 	 * 0.7598627805709839

F1_macro
only_threads_unlimit 	 0.7100909426490822 	 * 0.7430602110490377

# clinformative

accuracy
only_threads_unlimit 	 0.8610634803771973 	 * 0.8627787232398987

F1_macro
only_threads_unlimit 	 * 0.6965854277735466 	 0.671234421699656

# clm

In [21]:
for al in all_cols:
    print(al)
    print(get_best('only_threads_unlimited_30000_cl_' + al, 'F1_macro', config_param='embed_prevent', config_val=6, one=False))

claudience
[0.7807716001848144, 0.7798221191028616, 0.7780568835588688, 0.7770901072326837, 0.7743443947303008, 0.7743443947303008, 0.7725868770757682, 0.7688112477373448, 0.7670617543449914, 0.7660234981330518, 0.7660234981330518, 0.5434365133613254, 0.46342601787487586]
clpersuasive
[0.6810226620284373, 0.6782121512466394, 0.6712919824562444, 0.6711522965350524, 0.6683630091321129, 0.6667743538767396, 0.6536086465989733, 0.6408808582721627, 0.6391532790492958, 0.6347117794486216, 0.4606845513413506]
clsentiment
[0.42082525683271954, 0.4098506991729498, 0.4078378451734689, 0.40400866190474294, 0.39644522144522143, 0.3956820632242051, 0.3924596018465127, 0.39168788333334165, 0.39136811826271023, 0.3881645462689144, 0.2498841698841699]
clagreement
[0.7374859287054409, 0.725517890772128, 0.7254106164006053, 0.7235810905034166, 0.7177890016348674, 0.7019011760330963, 0.6964900583620273, 0.693516355140187, 0.6710874640882539, 0.6575471698113208, 0.4690346083788707]
cldisagreement
[0.726882

In [ ]:
lm_threads_cl_cltopi